In [ ]:
import joblib
import pandas as pd
import numpy as np

players = pd.read_csv("../data/players.csv")

player_rounds = pd.read_csv("../data/player_rounds.csv")

rounds = pd.read_csv("../data/rounds.csv")

valid_ids = pd.read_csv("../data/valid_ids.csv")

push_stats = pd.read_csv("../data/push_stats.csv")

teams = pd.read_csv("../data/teams.csv")

healspread = pd.read_csv("../data/healspread.csv")

info = pd.read_csv("../data/info.csv")

round_events = pd.read_csv('../data/round_events.csv')

koth_rounds = pd.read_csv('../data/koth_rounds.csv')


model_ready_data = joblib.load("../data/pkls/model_ready_data_dict.pkl")
X = model_ready_data['X']
players_wide  = model_ready_data['players_wide']

y = model_ready_data['y']
log_ids = joblib.load("../data/pkls/log_id/log_id.pkl")

In [17]:
players_wide

,id,team,demoman_Unnamed: 0,scout_pocket_Unnamed: 0,scout_roamer_Unnamed: 0,soldier_pocket_Unnamed: 0,soldier_roamer_Unnamed: 0,demoman_steamid,scout_pocket_steamid,scout_roamer_steamid,...,medic_avg_time_before_using,medic_avg_time_to_build,medic_avg_uber_length,medic_biggest_advantage_lost,medic_deaths_with_95_99_uber_rate,medic_deaths_within_20s_after_uber_rate,medic_drops_rate,medic_medic_deaths,medic_uberspm,winner
0,1459201,Blue,210193,210197,210191,210201,210200,[U:1:115215794],[U:1:62591040],[U:1:110693313],...,22.153846,49.076923,6.866667,12.0,NaN,0.0769,NaN,9.0,0.4319,1
1,1459201,Red,210195,210192,210196,210198,210194,[U:1:246637404],[U:1:111856333],[U:1:34673204],...,34.100000,63.307692,6.900000,40.0,0.1000,0.1000,0.3000,5.0,0.3322,0
2,1459213,Blue,210172,210179,210169,210171,210174,[U:1:193086127],[U:1:99217913],[U:1:119490553],...,22.333333,77.000000,7.350000,40.0,NaN,0.1667,0.1667,16.0,0.2249,1
3,1459213,Red,210177,210173,210175,210178,210176,[U:1:94390071],[U:1:196093068],[U:1:87138976],...,8.000000,64.750000,7.233333,10.0,0.2000,0.6000,0.4000,23.0,0.3807,0
4,1459233,Blue,210156,210166,210165,210167,210161,[U:1:102584616],[U:1:90669165],[U:1:84046000],...,35.090909,65.636364,6.809091,32.0,0.0909,0.2727,NaN,12.0,0.3498,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32645,3897345,Red,79411,79406,79408,79407,79410,[U:1:424357],[U:1:139578113],[U:1:151404046],...,71.400000,53.333333,6.950000,NaN,NaN,0.2000,NaN,1.0,0.3215,0
32646,3905691,Blue,67369,67359,67370,67368,67361,[U:1:93355936],[U:1:10403381],[U:1:93699014],...,50.272727,60.000000,6.945455,34.0,NaN,0.0909,NaN,2.0,0.4664,1
32647,3905691,Red,67363,67367,67364,67360,67362,[U:1:16284945],[U:1:87548232],[U:1:190301470],...,46.777778,59.666667,7.133333,NaN,0.1111,0.1111,NaN,7.0,0.3816,0
32648,3905701,Blue,10,0,11,9,2,[U:1:93355936],[U:1:10403381],[U:1:93699014],...,37.666667,58.700000,6.550000,18.0,NaN,0.3333,0.6667,12.0,0.2413,1


In [3]:
import pandas as pd
import numpy as np

def high_corr_pairs(corr_matrix, threshold=0.8, absolute=True):
    corr_matrix = corr_matrix.copy()
    # Mask upper triangle and diagonal
    mask = np.tril(np.ones(corr_matrix.shape), k=-1).astype(bool)
    corr_unstacked = corr_matrix.where(mask).stack()
    
    # Filter by threshold
    if absolute:
        corr_filtered = corr_unstacked[abs(corr_unstacked) >= threshold]
    else:
        corr_filtered = corr_unstacked[corr_unstacked >= threshold]
    
    # Format as dataframe
    return corr_filtered.reset_index().rename(
        columns={'level_0': 'var1', 'level_1': 'var2', 0: 'correlation'}
    )

corrs = high_corr_pairs(X.corr(), threshold=0.85)


In [4]:
from optbinning import BinningProcess
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

X_bin = X.drop('id',axis = 1).copy()

i = 0
problem = pd.Series()
for col in X_bin:
    if col == "demoman_dapm":
        continue
    binning_process = BinningProcess(
        variable_names=["demoman_dapm",col],
        categorical_variables=[],
        max_n_bins = 5,
        binning_fit_params={
            "default": {
                "solver": "cp",
                "max_n_bins": 5,
                "min_bin_size": 0.05
            }
        }
    )

    try:
         binning_process.fit(X_bin[['demoman_dapm',col]], y)
    except Exception as e:
        i += 1
        problem.loc[len(problem)] = col

(CVXPY) Aug 16 11:54:18 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.14.6206). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Aug 16 11:54:18 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.14.6206). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')


In [5]:
from optbinning import OptimalBinning
optb = OptimalBinning(name="demoman_dapd", dtype="numerical", solver="cp")
optb.fit(X['demoman_dapd'], y)

b = pd.Series(optb.transform(X['demoman_dapd'], metric = "bins"))



In [6]:
X_bin_no_error = X_bin.drop(problem,axis = 1).copy()

binning_process = BinningProcess(
    variable_names=[col for col in X_bin_no_error.columns],
    categorical_variables=[],
    max_n_bins = 10,
    min_bin_size= .05,
    
    binning_fit_params={
        "default": {
            "solver": "cp",
            "max_n_bins": 5,
            "min_bin_size": 0.05,
            "sample_weights": [1,23,4]
        }
    }
)
binning_process.fit(X_bin_no_error,y)

,variable_names,"['demoman_dapd', 'scout_pocket_dapd', ...]"
,max_n_prebins,20
,min_prebin_size,0.05
,min_n_bins,None
,max_n_bins,10
,min_bin_size,0.05
,max_bin_size,None
,max_pvalue,None
,max_pvalue_policy,'consecutive'
,selection_criteria,None
,fixed_variables,None


In [7]:
X_bins = binning_process.transform(X_bin_no_error, metric="bins")
X_exploded = pd.get_dummies(X_bins).copy()
X_exploded['win'] = y.copy()
X_exploded['id'] = X['id'].copy()
X_exploded = X_exploded.merge(info[['id','clean_map_name']])

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from pandas.errors import PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)

def conditional_target_encoding_vectorized(df, feature_cols, map_col='clean_map_name', target_col='win', id_col='id', n_splits=5, smoothing=5):
    """
    Fast k-fold leakage-free map-adjusted target encoding for binary feature columns (vectorized version).
    Preserves ID, map, and target columns for easy merging back.
    """
    # Initialize output with identifier columns
    df_encoded = df[[id_col, map_col, target_col]].copy()
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Prepare empty columns for map-adjusted features
    for feature in feature_cols:
        df_encoded[feature + '_map_adj'] = 0.0
    
    for train_idx, val_idx in kf.split(df):
        train, val = df.iloc[train_idx], df.iloc[val_idx]
        
        # Compute global feature-bin win rates for smoothing
        global_rates = train[feature_cols].multiply(train[target_col], axis=0).sum() / train[feature_cols].sum()
        
        for feature in feature_cols:
            # Only rows where feature is active
            train_feat = train[train[feature] == 1].copy()
            
            # Group by map and compute mean and count
            map_feat_stats = train_feat.groupby(map_col)[target_col].agg(['mean', 'count']).reset_index()
            map_feat_stats['smoothed'] = (
                (map_feat_stats['count'] * map_feat_stats['mean'] + smoothing * global_rates[feature]) / 
                (map_feat_stats['count'] + smoothing)
            )
            
            # Merge smoothed map-feature encoding with validation fold
            val_feat = val[[feature, map_col]].copy()
            val_feat = val_feat.merge(map_feat_stats[[map_col, 'smoothed']], on=map_col, how='left')
            
            # Fill missing maps with global rate
            val_feat['smoothed'] = val_feat['smoothed'].fillna(global_rates[feature])
            
            # Apply encoding only to active feature rows
            df_encoded.loc[val_idx, feature + '_map_adj'] = val_feat['smoothed'] * val_feat[feature]
    
    return df_encoded

# Example usage:
feature_cols = [col for col in X_exploded.columns if col not in ['id', 'clean_map_name', 'win']]
df_map_adjusted = conditional_target_encoding_vectorized(X_exploded, feature_cols, map_col='clean_map_name', target_col='win', id_col='id', n_splits=5)


In [9]:
from pandas.errors import PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)
final_df = pd.DataFrame()
for col in X_bin_no_error.columns:
        
    binned_cols = [bins for bins in df_map_adjusted.columns if bins.startswith(col)]
    collected_column = df_map_adjusted[binned_cols].T.sum()
    final_df[col + "_map_adj"] = collected_column.copy()

In [12]:
corrs = high_corr_pairs(final_df.corr(), threshold=0.85)


In [15]:
191 * 191

36481